<a href="https://colab.research.google.com/github/abhranil-datascience/ArtificialNeuralNetworkPOC/blob/master/ANNRegressionPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/DL/ANN/Regression')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
########## Import Dataset ############
import pandas as pd
dataset=pd.read_csv('50_Startups.csv')
X=dataset.iloc[:,0:4].values
Y=dataset.iloc[:,4].values
########## Categorical Variable #############
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le=LabelEncoder()
X[:,3]=le.fit_transform(X[:,3])
oe=OneHotEncoder(categorical_features=[3])
X=oe.fit_transform(X).toarray()
X=X[:,1:]
########## Train Test Split ################
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(X,Y,test_size=0.1,random_state=42)
########## Feature Scaling ##################
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
sc_Y=StandardScaler()
XTrainScaled=sc_X.fit_transform(XTrain)
XTestScaled=sc_X.transform(XTest)
YTrainScaled=sc_Y.fit_transform(YTrain.reshape(-1,1))
######### Check VIF ################
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=[0]*XTrainScaled.shape[1]
for col in range(0,XTrainScaled.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrainScaled,exog_idx=col)
VIF

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


[1.4412728272469775,
 1.3656302321577596,
 2.3995142500863085,
 1.1841346272790574,
 2.384157782372035]

In [0]:
######### ANN ##############
filepath="bestregressionmodel.h5"
from keras.models import Sequential, load_model
from keras.layers import Dense,Dropout
from keras.callbacks import ModelCheckpoint
save_best_model=ModelCheckpoint(filepath, monitor='mean_absolute_error', verbose=1, save_best_only=True, mode='min')
RegressorANN=Sequential()
RegressorANN.add(Dense(input_shape=(5,),units=1024,activation='relu',kernel_initializer='random_uniform'))
RegressorANN.add(Dropout(0.2))
RegressorANN.add(Dense(units=1024,activation='relu',kernel_initializer='random_uniform'))
RegressorANN.add(Dense(units=1,kernel_initializer='random_uniform'))
RegressorANN.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
RegressorANN.fit(x=XTrainScaled,y=YTrainScaled,batch_size=2,epochs=500,callbacks=[save_best_model])

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Epoch 1/500





45/45 [==============================] - 5s 111ms/step - loss: 0.3217 - mean_absolute_error: 0.4670

Epoch 00001: mean_absolute_error improved from inf to 0.46702, saving model to bestregressionmodel.h5
Epoch 2/500
45/45 [==============================] - 0s 2ms/step - loss: 0.1891 - mean_absolute_error: 0.3098

Epoch 00002: mean_absolute_error improved from 0.46702 to 0.30977, saving model to bestregressionmodel.h5
Epoch 3/500
45/45 [==============================] - 0s 2ms/step - loss: 0.0699 - mean_absolute_error: 0.2055

Epoch 00003: mean_absolute_error improved from 0.30977 to 0.20548, saving model to bestregressionmodel.h5
Epoch 4/500
45/45 [==============================] - 0s 2ms/step - loss: 0.0514 - mean_absolute_error: 0.1797

Epoch 00004: mean_absolute_error improved from 0.20548 to 0.17970, saving model to bestregressionmodel.h5
Epoch 

In [0]:
######## Load Best Model and Predict #############
bestRegModel=load_model('bestregressionmodel.h5')
YPredbestmodel=sc_Y.inverse_transform(bestRegModel.predict(XTestScaled))
from sklearn import metrics as mt
print("Goodness of Fit for BestModel:"+str(mt.r2_score(YTest,YPredbestmodel)))

Goodness of Fit for BestModel:0.9698927489321916
